In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split 

from sklearn.preprocessing import PolynomialFeatures

from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor\

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import VotingRegressor

In [3]:
#%pip install category_encoders

In [4]:
path_train = 'train.csv' # содержит только имя файла, без имен папок !!!
df = pd.read_csv(path_train)
df

,company,model,year,condition,mileage(km),fuel,volume(cm3),color,transmission,drive_unit,vehicle_size_class,priceUSD
0,renault,laguna,2000,with mileage,230000.0,petrol,1800.0,blue,mechanics,front-wheel drive,D,2300
1,chrysler,pacifica,2005,with mileage,310000.0,petrol,3500.0,black,auto,front-wheel drive,NaN,6300
2,mercedes-benz,c-klass,2010,with mileage,115000.0,petrol,1800.0,gray,auto,rear drive,D,11200
3,hyundai,accent,2017,with mileage,45500.0,petrol,1400.0,gray,auto,front-wheel drive,B,10886
4,renault,kadjar,2016,with mileage,139000.0,diesel,1500.0,red,auto,front-wheel drive,J,15300
...,...,...,...,...,...,...,...,...,...,...,...,...
38000,renault,laguna,1999,with mileage,314000.0,petrol,1600.0,green,mechanics,front-wheel drive,D,1999
38001,opel,insignia,2010,with mileage,230000.0,petrol,1600.0,black,mechanics,front-wheel drive,D,7606
38002,kia,optima,2012,with mileage,115000.0,petrol,2400.0,black,mechanics,front-wheel drive,D,11850
38003,volkswagen,vento,1994,with mileage,300000.0,diesel,1900.0,white,mechanics,front-wheel drive,NaN,1200


In [5]:
df.info()

df.drop_duplicates(inplace=True)#vehicle_size_class проблемная колонка с на
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38005 entries, 0 to 38004
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   company             38005 non-null  object 
 1   model               38005 non-null  object 
 2   year                38005 non-null  int64  
 3   condition           38005 non-null  object 
 4   mileage(km)         38005 non-null  float64
 5   fuel                38005 non-null  object 
 6   volume(cm3)         38005 non-null  float64
 7   color               38005 non-null  object 
 8   transmission        38005 non-null  object 
 9   drive_unit          38005 non-null  object 
 10  vehicle_size_class  34434 non-null  object 
 11  priceUSD            38005 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 3.5+ MB


(37966, 12)

In [6]:
cat_features=['company', 'model', 'condition', 'fuel', 'color', 'transmission', 'drive_unit', 'vehicle_size_class']
num_features=['year', 'mileage(km)', 'volume(cm3)']

In [7]:
X = df.drop(columns='priceUSD')
y = df.priceUSD

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                     test_size=0.3, random_state=0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)# соразмерны
print(y_test.shape)

(26576, 11)
(26576,)
(11390, 11)
(11390,)


In [9]:
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[('ce', ce.OrdinalEncoder()),
           ('scaler', MinMaxScaler())]
)

size_map = [{
    'col': 'vehicle_size_class',
    'mapping': {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'S': 7, 'M': 8, 'J': 9}
}]

clarity_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy='most_frequent')),
    ('ce', ce.OrdinalEncoder(mapping=size_map)),
    ('scaler', MinMaxScaler())
])    


CT = ColumnTransformer([
       ("num", numeric_transformer, num_features),
    ("cat", categorical_transformer, cat_features),
        ("ordinal_map", clarity_transformer, ['vehicle_size_class'])
        ]).set_output(transform='pandas')

display(CT)
ct = CT.fit_transform(X_train)


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['year', 'mileage(km)', 'volume(cm3)']),
                                ('cat',
                                 Pipeline(steps=[('ce', OrdinalEncoder()),
                                                 ('scaler', MinMaxScaler())]),
                                 ['company', 'model', 'condition', 'fuel',
                                  'color', 'transmission', 'drive_unit',
                                  'vehicle_size_class']),
                                ('ordinal_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['vehicle_size_class'])])

In [19]:
pipe = Pipeline([
                ('model', TransformedTargetRegressor(
                        regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator',  RandomForestRegressor(n_estimators=1000))
                            ]),
                        transformer=PowerTransformer(),
                        check_inverse = False)
                )])


pipe.fit(X_train, y_train)

Pipeline(steps=[('model',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=Pipeline(steps=[('preproc',
                                                                       ColumnTransformer(transformers=[('num',
                                                                                                        Pipeline(steps=[('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['year',
                                                                                                         'mileage(km)',
                                                                                                         'volume(cm3)']),
                                                                                                       ('cat',
                                                                                                        Pipeline(steps=[('ce',
                                                                                                                         OrdinalEncoder()),
                                                                                                                        ('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['company',
                                                                                                         'model',
                                                                                                         'condition',
                                                                                                         'fuel',
                                                                                                         'color'...
                                                                                                       ('ordinal_map',
                                                                                                        Pipeline(steps=[('missing_num',
                                                                                                                         SimpleImputer(strategy='most_frequent')),
                                                                                                                        ('ce',
                                                                                                                         OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                                                                                  'data_type': dtype('O'),
                                                                                                                                                  'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                                                                        ('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['vehicle_size_class'])])),
                                                                      ('estimator',
                                                                       RandomForestRegressor(n_estimators=1000))]),
                                            transformer=PowerTransformer()))])

In [20]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, pipe.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, pipe.predict(X_test))))

Правильность на обучающем наборе: 0.077
Правильность на тестовом наборе: 0.225


In [ ]:
# pipe = Pipeline([('preprocessing', CT), 
#                  ('regressor', svm.SVC())])

# C=np.array([0.02,0.03,0.04,0.05, 0.06,0.07])
# gamma=[0.06,0.07,0.1,1.1,1.2,1.3]
# shuffle_split = ShuffleSplit(test_size = .3, train_size = .6, n_splits = 5, random_state = 42)


# param_grid =[
#     {'preprocessing': [CT], 
#      'regressor': [svm.SVR()],
#      'regressor__C': C,
#      'regressor__gamma': gamma
#      }
# ]
# grid = GridSearchCV(pipe, param_grid, return_train_score=True,scoring='neg_mean_absolute_percentage_error',cv=shuffle_split)
# grid.fit(X_train,y_train)
# table=pd.DataFrame(grid.cv_results_)
# pv_table=table.pivot_table(index='param_classifier__C',columns='param_classifier__gamma',values='mean_test_score')
# pv_table
# f, ax = plt.subplots(figsize=(9, 6))
# sns.heatmap(pv_table, annot=True, linewidths=.5, ax=ax);

In [ ]:
# pipe = Pipeline([
#                 ('model', TransformedTargetRegressor(
#                         regressor = Pipeline(steps=[
#                                 ('preproc', CT),
#                                 ('estimator',  Ridge())
#                             ]),
#                         transformer=PowerTransformer(),
#                         check_inverse = False)
#                 )])


# pipe.fit(X_train, y_train)

In [ ]:
# print('тренировочный MAPE: {:.3f}'.format( mean_absolute_percentage_error(y_test, pipe.predict(X_test) ) ))

In [24]:
pipe=TransformedTargetRegressor(
                regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator', Ridge())
                            ]),
                transformer=QuantileTransformer(n_quantiles=1000, output_distribution="normal")
)
param_grid =[
    {'regressor__estimator': [Ridge()],
     'regressor__preproc':[CT],
#'regressor__preproc__axis_transformer__polynom__degree':[2,3,4],
     'regressor__estimator__alpha':[0.000000001, 0.01,0.1,0.5,1.5],
     'regressor__estimator__solver':[ 'lsqr', 'sag', 'saga', 'lbfgs']
     
     }]
grid = GridSearchCV(pipe, n_jobs=-1, param_grid=param_grid, scoring='neg_mean_absolute_percentage_error')
grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/compose/_target.py", line 262, in fit
    self.regressor_.fit(X, y_trans, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/

GridSearchCV(estimator=TransformedTargetRegressor(regressor=Pipeline(steps=[('preproc',
                                                                             ColumnTransformer(transformers=[('num',
                                                                                                              Pipeline(steps=[('scaler',
                                                                                                                               MinMaxScaler())]),
                                                                                                              ['year',
                                                                                                               'mileage(km)',
                                                                                                               'volume(cm3)']),
                                                                                                             ('cat',
                                                                                                              Pipeline(steps=[('ce',
                                                                                                                               OrdinalEncoder()),
                                                                                                                              ('scaler',
                                                                                                                               MinMaxScaler())]),
                                                                                                              ['company',
                                                                                                               'model',
                                                                                                               'condition',
                                                                                                               'fuel',
                                                                                                               'color',
                                                                                                               'transmission',
                                                                                                               'drive...
                                                                                   'drive_unit',
                                                                                   'vehicle_size_class']),
                                                                                 ('ordinal_map',
                                                                                  Pipeline(steps=[('missing_num',
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('ce',
                                                                                                   OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                                                            'data_type': dtype('O'),
                                                                                                                            'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                                                  ('scaler',
                                                                                                   MinMaxScaler())]),
                                                                                  ['vehicle_size_class'])])]}],
             scoring='neg_mean_absolute_percentage_error')

In [25]:
print("Наилучшие параметры:" ,grid.best_params_)
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах:" ,grid.best_score_)
print("Правильность для наилучшей модели на тестовом наборе:",grid.score(X_test, y_test))

Наилучшие параметры: {'regressor__estimator': Ridge(alpha=0.01, solver='saga'), 'regressor__estimator__alpha': 0.01, 'regressor__estimator__solver': 'saga', 'regressor__preproc': ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', MinMaxScaler())]),
                                 ['year', 'mileage(km)', 'volume(cm3)']),
                                ('cat',
                                 Pipeline(steps=[('ce', OrdinalEncoder()),
                                                 ('scaler', MinMaxScaler())]),
                                 ['company', 'model', 'condition', 'fuel',
                                  'color', 'transmission', 'drive_unit',
                                  'vehicle_size_class']),
                                ('ordinal_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                       

In [ ]:
# model_rf = Pipeline(steps=[
#     ('preproc', CT),
#     ('estimator', RandomForestRegressor())
# ])
# shuffle_split = ShuffleSplit(test_size = .3, train_size = .6, n_splits = 5, random_state = 42)
# param_grid = {
#     'preproc':[CT],
#     'estimator__criterion': ['squared_error', 'absolute_error'],
#     'estimator__n_estimators': [1000],
#     'estimator__max_features':['sqrt', None,'log2'],
#   'estimator__min_samples_leaf':np.arange(1,5)
    
    
# }

# # Определите модель Pipeline с преобразованием данных и классификатором

# grid_rf = GridSearchCV(model_rf, param_grid=param_grid, scoring='neg_mean_absolute_percentage_error',cv=shuffle_split)
# grid_rf.fit(X_train, y_train)

In [ ]:
# print("Наилучшие параметры:" ,grid_rf.best_params_)
# print("Средняя правильность для наилучшей модели кроссвалидации на" 
#       "валидационных тестовых наборах:" ,grid_rf.best_score_)
# print("Правильность для наилучшей модели на тестовом наборе:",grid_rf.score(X_test, y_test))

In [ ]:
# pipe = Pipeline([
#                 ('model', TransformedTargetRegressor(
#                         regressor = Pipeline(steps=[
#                                 ('preproc', CT),
#                                 ('estimator',  RandomForestRegressor(n_estimators=1000,criterion= 'absolute_error', max_features= None, min_samples_leaf=3))
#                             ]),
#                         transformer=PowerTransformer(),
#                         check_inverse = False)
#                 )])


# pipe.fit(X_train, y_train)

In [ ]:
# print('тренировочный MAPE: {:.3f}'.format( mean_absolute_percentage_error(y_test, pipe.predict(X_test) ) ))

In [10]:
pipe_knn = Pipeline([('preprocessing',CT), 
                    ('classifier',    RandomForestRegressor(n_estimators=1000,criterion= 'squared_error', max_features= None, min_samples_leaf=3))])
pipe_gbrt = Pipeline([('preprocessing',CT), 
                    ('classifier',  RandomForestRegressor(n_estimators=1000))])

In [11]:
pipe_knn.fit(X_train, y_train)
pipe_gbrt.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['year', 'mileage(km)',
                                                   'volume(cm3)']),
                                                 ('cat',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['company', 'model',
                                                   'condition', 'fuel', 'color',
                                                   'transmission', 'drive_unit',
                                                   'vehicle_size_class']),
                                                 ('ordinal_map',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['vehicle_size_class'])])),
                ('classifier', RandomForestRegressor(n_estimators=1000))])

In [15]:
eclf2 = VotingRegressor(
    estimators=[ ('rf', pipe_knn), ('knn', pipe_gbrt)], weights=[11,4])

In [16]:
eclf2.fit(X_train, y_train)

VotingRegressor(estimators=[('rf',
                             Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('scaler',
                                                                                                MinMaxScaler())]),
                                                                               ['year',
                                                                                'mileage(km)',
                                                                                'volume(cm3)']),
                                                                              ('cat',
                                                                               Pipeline(steps=[('ce',
                                                                                                OrdinalEncoder()),
                                                                                               ('scaler',
                                                                                                MinMaxScaler())]),
                                                                               ['company',
                                                                                'model',
                                                                                'condition',
                                                                                'fuel',
                                                                                'color',
                                                                                'transmission',
                                                                                'drive_unit',
                                                                                'vehicle_size...
                                                                                'vehicle_size_class']),
                                                                              ('ordinal_map',
                                                                               Pipeline(steps=[('missing_num',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('ce',
                                                                                                OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                                                         'data_type': dtype('O'),
                                                                                                                         'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                                               ('scaler',
                                                                                                MinMaxScaler())]),
                                                                               ['vehicle_size_class'])])),
                                             ('classifier',
                                              RandomForestRegressor(n_estimators=1000))]))],
                weights=[11, 4])

In [17]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, eclf2.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, eclf2.predict(X_test))))

Правильность на обучающем наборе: 0.146
Правильность на тестовом наборе: 0.254


In [10]:
pipe_knn = Pipeline([('preprocessing',CT), 
                    ('classifier',    RandomForestRegressor(n_estimators=1000,criterion= 'squared_error', max_features= None, min_samples_leaf=3))])

In [11]:
pipe_knn.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['year', 'mileage(km)',
                                                   'volume(cm3)']),
                                                 ('cat',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['company', 'model',
                                                   'condition', 'fuel', 'color',
                                                   'transmission', 'drive_unit',
                                                   'vehicle_size_class']),
                                                 ('ordinal_map',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['vehicle_size_class'])])),
                ('classifier',
                 RandomForestRegressor(max_features=None, min_samples_leaf=3,
                                       n_estimators=1000))])

In [12]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, pipe_knn.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, pipe_knn.predict(X_test))))

Правильность на обучающем наборе: 0.163
Правильность на тестовом наборе: 0.253


In [13]:
pipe_knn = Pipeline([('preprocessing',CT), 
                    ('classifier',    RandomForestRegressor(n_estimators=1000))])

In [14]:
pipe_knn.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['year', 'mileage(km)',
                                                   'volume(cm3)']),
                                                 ('cat',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['company', 'model',
                                                   'condition', 'fuel', 'color',
                                                   'transmission', 'drive_unit',
                                                   'vehicle_size_class']),
                                                 ('ordinal_map',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['vehicle_size_class'])])),
                ('classifier', RandomForestRegressor(n_estimators=1000))])

In [15]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, pipe_knn.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, pipe_knn.predict(X_test))))

Правильность на обучающем наборе: 0.099
Правильность на тестовом наборе: 0.258


In [10]:
model_rf = Pipeline(steps=[
    ('preproc', CT),
    ('estimator', RandomForestRegressor())
])
shuffle_split = ShuffleSplit(test_size = .3, train_size = .6, n_splits = 5, random_state = 42)
param_grid = {
    'preproc':[CT],
    # 'estimator__criterion': ['squared_error'],
    'estimator__n_estimators': [1000],
    'estimator__max_features':['sqrt', None,'log2'],
  'estimator__min_samples_leaf':np.arange(6,9)
    
    
}

# Определите модель Pipeline с преобразованием данных и классификатором

grid_rf = GridSearchCV(model_rf, n_jobs=-1, param_grid=param_grid, scoring='neg_mean_absolute_percentage_error',cv=shuffle_split)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=42, test_size=0.3, train_size=0.6),
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['year',
                                                                          'mileage(km)',
                                                                          'volume(cm3)']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('ce',
                                                                                          OrdinalEncoder()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['company',
                                                                          'model',
                                                                          'condition',...
                                                                       'transmission',
                                                                       'drive_unit',
                                                                       'vehicle_size_class']),
                                                                     ('ordinal_map',
                                                                      Pipeline(steps=[('missing_num',
                                                                                       SimpleImputer(strategy='most_frequent')),
                                                                                      ('ce',
                                                                                       OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                                                'data_type': dtype('O'),
                                                                                                                'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                                      ('scaler',
                                                                                       MinMaxScaler())]),
                                                                      ['vehicle_size_class'])])]},
             scoring='neg_mean_absolute_percentage_error')

In [11]:
print("Наилучшие параметры:" ,grid_rf.best_params_)
print("Средняя правильность для наилучшей модели кроссвалидации на" 
      "валидационных тестовых наборах:" ,grid_rf.best_score_)
print("Правильность для наилучшей модели на тестовом наборе:",grid_rf.score(X_test, y_test))

Наилучшие параметры: {'estimator__max_features': None, 'estimator__min_samples_leaf': 6, 'estimator__n_estimators': 1000, 'preproc': ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', MinMaxScaler())]),
                                 ['year', 'mileage(km)', 'volume(cm3)']),
                                ('cat',
                                 Pipeline(steps=[('ce', OrdinalEncoder()),
                                                 ('scaler', MinMaxScaler())]),
                                 ['company', 'model', 'condition', 'fuel',
                                  'color', 'transmission', 'drive_unit',
                                  'vehicle_size_class']),
                                ('ordinal_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
             

In [12]:
pipe_knn = Pipeline([('preprocessing',CT), 
                    ('classifier',    RandomForestRegressor(n_estimators=1000))])

In [13]:
pipe_knn.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['year', 'mileage(km)',
                                                   'volume(cm3)']),
                                                 ('cat',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['company', 'model',
                                                   'condition', 'fuel', 'color',
                                                   'transmission', 'drive_unit',
                                                   'vehicle_size_class']),
                                                 ('ordinal_map',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['vehicle_size_class'])])),
                ('classifier', RandomForestRegressor(n_estimators=1000))])

In [14]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, pipe_knn.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, pipe_knn.predict(X_test))))

Правильность на обучающем наборе: 0.099
Правильность на тестовом наборе: 0.257


In [16]:
pipe_knn_B = Pipeline([('preprocessing',CT), 
                    ('classifier',    RandomForestRegressor())])

In [17]:
pipe_knn_B.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['year', 'mileage(km)',
                                                   'volume(cm3)']),
                                                 ('cat',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['company', 'model',
                                                   'condition', 'fuel', 'color',
                                                   'transmission', 'drive_unit',
                                                   'vehicle_size_class']),
                                                 ('ordinal_map',
                                                  Pipeline(steps=[('missing_num',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['vehicle_size_class'])])),
                ('classifier', RandomForestRegressor())])

In [18]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, pipe_knn_B.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, pipe_knn_B.predict(X_test))))

Правильность на обучающем наборе: 0.099
Правильность на тестовом наборе: 0.259


In [10]:
pipe = Pipeline([
                ('model', TransformedTargetRegressor(
                        regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator',  RandomForestRegressor(n_estimators=1000,criterion='friedman_mse'))
                            ]),
                        transformer=PowerTransformer(),
                        check_inverse = False)
                )])


pipe.fit(X_train, y_train)

Pipeline(steps=[('model',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=Pipeline(steps=[('preproc',
                                                                       ColumnTransformer(transformers=[('num',
                                                                                                        Pipeline(steps=[('scaler',
                                                                                                                         StandardScaler())]),
                                                                                                        ['year',
                                                                                                         'mileage(km)',
                                                                                                         'volume(cm3)']),
                                                                                                       ('cat',
                                                                                                        Pipeline(steps=[('ce',
                                                                                                                         OrdinalEncoder()),
                                                                                                                        ('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['company',
                                                                                                         'model',
                                                                                                         'condition',
                                                                                                         'fuel',
                                                                                                         'colo...
                                                                                                        Pipeline(steps=[('missing_num',
                                                                                                                         SimpleImputer(strategy='most_frequent')),
                                                                                                                        ('ce',
                                                                                                                         OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                                                                                  'data_type': dtype('O'),
                                                                                                                                                  'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                                                                        ('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['vehicle_size_class'])])),
                                                                      ('estimator',
                                                                       RandomForestRegressor(criterion='friedman_mse',
                                                                                             n_estimators=1000))]),
                                            transformer=PowerTransformer()))])

In [11]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, pipe.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, pipe.predict(X_test))))

Правильность на обучающем наборе: 0.077
Правильность на тестовом наборе: 0.224


### Выполняется подбора леса. тк код считается долго решетка срезана

In [8]:
model_rf = Pipeline(steps=[
    ('preproc', CT),
    ('estimator', RandomForestRegressor())
])
shuffle_split = ShuffleSplit(test_size = .3, train_size = .6, n_splits = 5, random_state = 42)
param_grid = {
    'preproc':[CT],
    'estimator__min_samples_split':[3,4],
    'estimator__max_depth':[10],
    
    # 'estimator__criterion': ['squared_error'],
    'estimator__n_estimators': [1000],
    'estimator__max_features':['sqrt', None,'log2'],
  'estimator__min_samples_leaf':np.arange(2,4)
    
    
}

# Определите модель Pipeline с преобразованием данных и классификатором

grid_rf = GridSearchCV(model_rf, n_jobs=-1, param_grid=param_grid, scoring='neg_mean_absolute_percentage_error',cv=shuffle_split)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=42, test_size=0.3, train_size=0.6),
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['year',
                                                                          'mileage(km)',
                                                                          'volume(cm3)']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('ce',
                                                                                          OrdinalEncoder()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['company',
                                                                          'model',
                                                                          'condition',...
                                                                       'transmission',
                                                                       'drive_unit',
                                                                       'vehicle_size_class']),
                                                                     ('ordinal_map',
                                                                      Pipeline(steps=[('missing_num',
                                                                                       SimpleImputer(strategy='most_frequent')),
                                                                                      ('ce',
                                                                                       OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                                                'data_type': dtype('O'),
                                                                                                                'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                                      ('scaler',
                                                                                       MinMaxScaler())]),
                                                                      ['vehicle_size_class'])])]},
             scoring='neg_mean_absolute_percentage_error')

In [9]:
print('Правильность на обучающем наборе: {:.3f}'.format(mean_absolute_percentage_error(y_train, grid_rf.predict(X_train))))

print('Правильность на тестовом наборе: {:.3f}'.format(mean_absolute_percentage_error(y_test, grid_rf.predict(X_test))))

Правильность на обучающем наборе: 0.281
Правильность на тестовом наборе: 0.300
